In [3]:
pip install pandas numpy scikit-learn nltk

In [4]:
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report


In [5]:
import nltk as nltk

Load the Whisky Dataset

In [6]:
import os

reqd_folder_path = os.path.join(os.getcwd(),'datasets')
print(reqd_folder_path)

dataset_file_path = os.path.join(reqd_folder_path,'indian_whisky_dataset_100.csv')
print(dataset_file_path)

d:\sandbox\python\experiments\machine-learning-experiments\Project-Machine-Learning\datasets
d:\sandbox\python\experiments\machine-learning-experiments\Project-Machine-Learning\datasets\indian_whisky_dataset_100.csv


In [7]:
temp_df = pd.read_csv(dataset_file_path)
print(temp_df['Brand Name'][0:20])

0             Royal Stag
1         Blenders Pride
2          Imperial Blue
3        McDowell's No.1
4         Antiquity Blue
5              Signature
6       Rockford Reserve
7             Peter Scot
8        Royal Challenge
9       Officer's Choice
10                  8 PM
11              Bagpiper
12    Director’s Special
13            Red Knight
14            After Dark
15          White & Blue
16        Golden Chariot
17              Haywards
18         Mansion House
19             Black Dog
Name: Brand Name, dtype: object


In [8]:
df = pd.read_csv(dataset_file_path)
print(df.head())

        Brand Name                       Tasting Notes  Alcohol Content (%)  \
0       Royal Stag     cinnamon, woody, oak, chocolate                 42.2   
1   Blenders Pride           citrus, oak, spicy, peaty                 40.2   
2    Imperial Blue         spicy, oak, honey, cinnamon                 42.5   
3  McDowell's No.1  floral, peaty, chocolate, cinnamon                 45.7   
4   Antiquity Blue    chocolate, nutty, spicy, caramel                 45.6   

   Price (INR)  
0         1964  
1         2681  
2         2981  
3         2503  
4         4536  


In [9]:
df

,Brand Name,Tasting Notes,Alcohol Content (%),Price (INR)
0,Royal Stag,"cinnamon, woody, oak, chocolate",42.2,1964
1,Blenders Pride,"citrus, oak, spicy, peaty",40.2,2681
2,Imperial Blue,"spicy, oak, honey, cinnamon",42.5,2981
3,McDowell's No.1,"floral, peaty, chocolate, cinnamon",45.7,2503
4,Antiquity Blue,"chocolate, nutty, spicy, caramel",45.6,4536
...,...,...,...,...
95,Sovereign Malt,"floral, cinnamon, peaty, buttery",43.1,3522
96,Glen Royale,"woody, fruity, oak, citrus",41.2,2252
97,Cask Master,"smoky, peaty, cinnamon, fruity",43.4,3182
98,Crestwood Select,"chocolate, fruity, caramel, honey",45.3,4556


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Brand Name           100 non-null    object 
 1   Tasting Notes        100 non-null    object 
 2   Alcohol Content (%)  100 non-null    float64
 3   Price (INR)          100 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ KB


Preprocess the Data

In [11]:
print(df.isnull().sum())

Brand Name             0
Tasting Notes          0
Alcohol Content (%)    0
Price (INR)            0
dtype: int64


In [ ]:
df.fillna(df.mode().iloc[0], inplace=True)

In [13]:
df

,Brand Name,Tasting Notes,Alcohol Content (%),Price (INR)
0,Royal Stag,"cinnamon, woody, oak, chocolate",42.2,1964
1,Blenders Pride,"citrus, oak, spicy, peaty",40.2,2681
2,Imperial Blue,"spicy, oak, honey, cinnamon",42.5,2981
3,McDowell's No.1,"floral, peaty, chocolate, cinnamon",45.7,2503
4,Antiquity Blue,"chocolate, nutty, spicy, caramel",45.6,4536
...,...,...,...,...
95,Sovereign Malt,"floral, cinnamon, peaty, buttery",43.1,3522
96,Glen Royale,"woody, fruity, oak, citrus",41.2,2252
97,Cask Master,"smoky, peaty, cinnamon, fruity",43.4,3182
98,Crestwood Select,"chocolate, fruity, caramel, honey",45.3,4556


Encode categorical variables

In [14]:
from sklearn.preprocessing import LabelEncoder


label_encoders = {}

for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

print(df.head())    




   Brand Name  Tasting Notes  Alcohol Content (%)  Price (INR)
0          43             25                 42.2         1964
1           8             29                 40.2         2681
2          25             86                 42.5         2981
3          30             36                 45.7         2503
4           4             19                 45.6         4536


Feature Selection & Normalization

In [15]:
from sklearn.preprocessing import StandardScaler

feature_columns = [col for col in df.columns if col!='Brand Name']
X = df[feature_columns]





In [16]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [17]:
X_scaled_df = pd.DataFrame(X_scaled, columns=feature_columns)


In [19]:
y = df['Brand Name']

Train the SVM Model

In [20]:
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_scaled_df,y)


SVC(kernel='linear', probability=True)

 Recommend a Similar Whiskey Brand

In [21]:
import numpy as np

def recommend_whisky(chosen_whisky):

    encoded_chosen = label_encoders['Brand Name'].transform([chosen_whisky])[0]
    # print(f'encoded chosen: {encoded_chosen}')

    whisky_index = df[df['Brand Name'] == encoded_chosen].index[0]
    
    # print(f'whisky_index: {whisky_index}')

    whisky_features = X_scaled_df.iloc[whisky_index].values.reshape(1, -1)
    # print(f'whisky_features: {whisky_features}')

    from sklearn.metrics.pairwise import cosine_similarity
    similiarities = cosine_similarity(whisky_features, X_scaled_df)
    # print(f'similiarities: {similiarities}')
    similiar_index = np.argsort(similiarities[0])[-2]
    # print(f'similiar_index: {similiar_index}')


    recommended_encoded = df.iloc[similiar_index]['Brand Name']
    recommended_encoded = int(recommended_encoded)
    # print(f'recommended_encoded: {recommended_encoded}')

    recommended_brand = label_encoders['Brand Name'].inverse_transform([recommended_encoded])
    # print(f'recommended_brand: {recommended_brand}')
    


    return recommended_brand



Now predict similiar brand

In [22]:
chosen_whisky = 'Signature'
recommended = recommend_whisky(chosen_whisky)

print(f'If you like {chosen_whisky}, you might also enjoy {recommended}')

If you like Signature, you might also enjoy ['Noble Spirit']


Brilliant!


Now, lets see if we can add a dropdown list for the whisky brands. So things get easier.

In [23]:
chosen_whisky2 = '8 PM'
recommended = recommend_whisky(chosen_whisky2)

print(f'If you like {chosen_whisky2}, you might also enjoy {recommended}')

If you like 8 PM, you might also enjoy ['Antiquity Blue']


Trying ipywidgets

In [24]:
import ipywidgets as widgets
from IPython.display import display


mydropdown = widgets.Dropdown(
    options = ['Signature', '8 PM', 'Blenders Pride', 'Royal Challenge','After Dark','Black Dog' ],
    value='Signature',
    description='Select your brand of whisky',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style={'description_width':'300px'}
    )


display(mydropdown)

def get_selected_value():
    return mydropdown.value

print(f'Selected value: {get_selected_value()}')




Dropdown(description='Select your brand of whisky', layout=Layout(width='500px'), options=('Signature', '8 PM'…

Selected value: Signature


In [26]:
chosen_whisky3 = get_selected_value()
recommended = recommend_whisky(chosen_whisky3)

print(f'If you like {chosen_whisky3}, you might also enjoy {recommended}')

If you like Royal Challenge, you might also enjoy ['Director’s Special']


-----------------------

We have successfully created an Artificial Intelligence model using Support Vector Machine that reccommends a brand of Whiskey from your whisky preferences.

Project successfully completed